In [1]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '31044' (I am process '25032')
1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cudnn.h"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 #if PY_MAJOR_VERSION >= 3
10 #define PyInt_FromLong PyLong_FromLong
11 #endif
12 
13 
14     namespace {
15     struct __struct_compiled_op_3442132e74e76f61361fb1c73112eef4 {
16         PyObject* __ERROR;
17 
18         PyObject* storage_V1;
19         
20 
21         __struct_compiled_op_3442132e74e76f61361fb1c73112eef4() {
22             // This is only somewhat safe because we:
23             //  1) Are not a vi

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.



['nvcc', '-shared', '-O3', '-Xlinker', '-rpath,/usr/local/cuda/lib64', '-m64', '-Xcompiler', '-fno-math-errno,-Wno-unused-label,-Wno-unused-variable,-Wno-write-strings,-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden', '-Xlinker', '-rpath,/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/cuda_ndarray', '-I/usr/local/cuda/include', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/numpy/core/include', '-I/home/asaeed9/anaconda2/include/python2.7', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda', '-L/home/asaeed9/anaconda2/lib', '-o', '/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/tmpdyR0ox/3442132e74e76f61361fb1c73112eef4.so', 'mod.cu', '-lcudnn', '-lpython2.7', '-lcudart']


In [ ]:
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys
from shutil import copyfile

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

In [ ]:
batch_size=64

In [ ]:
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data

In [ ]:
%cd ../data/2cat/train

In [ ]:
#copy training images
def copy_samples(n):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(n): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(500): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd /home/asaeed9/work/data/2cat/train

In [ ]:
def clean_up():
    %rm $path/train/cats/*
    %rm $path/train/dogs/*
    %rm $path/valid/cats/*
    %rm $path/valid/dogs/*

In [ ]:
clean_up()

In [ ]:
%cd /home/asaeed9/work/data/2cat/train
#for n in range(1000, 5001, 1000):
n = 1500
print('sample size: {}'.format(n))
copy_samples(n)

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
#     model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
#                      nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)

In [ ]:
conv1(tr_batches)

In [ ]:
results_path

In [ ]:
mod.save_weights(results_path+ '/' +'ft_15000.e10')